<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW05_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

part 1 load file (finish)
part 2 clean data (finish on 12/5)
part 3 tokenize : sentencepiece (finish on 12/6)
part 4 build model encoder decoder (finish on 12/7)
part 5 beam search (finish on 12/8)
part 6 label smooth (finish on 12/9)

part 7 test (finish on 12/10)


In [164]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copyfile('/content/drive/MyDrive/ted2020.tgz','/content/ted2020.tgz')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/ted2020.tgz'

In [165]:
import tarfile

rawdata_file = "/content/ted2020.tgz"
unzip_path = "/content/train_dev/"
# open file
file = tarfile.open(rawdata_file)

# extracting file
file.extractall(unzip_path)

file.close()


In [166]:
src_lang = "raw.en"
tgt_lang = "raw.zh"

src_path = f"{unzip_path}{src_lang}"
tgt_path = f"{unzip_path}{tgt_lang}"
print(tgt_path)

/content/train_dev/raw.zh


In [261]:
print(chr(10022).join([chr(21407), chr(31070), chr(21855), chr(21205)]))

原✦神✦啟✦動


In [ ]:
# for path in [src_path,tgt_path]:
#   with open(path, "r") as f:
  #  data = f.readlines()
  #  print(type(data),data[0:5])#<class 'list'> ['Thank you so much, Chris.\n', "And it'......on.\n']
    # data = f.read().splitlines()
    # print(type(data),data[0:5],len(data))

In [ ]:
# import re
# print(ord("，"))
# print(chr(65292))

In [167]:
import unicodedata
def to_halfwidth(string):
  return "".join(unicodedata.normalize('NFKC',letter) for letter in string)
# print(to_halfwidth("ＡＳＤＦＦＦＦＱ")) #ASDFFFFQ

In [168]:
import string
import re
def clean_s_zh(s):
    s = to_halfwidth(s)
    # step 1 : delete — _
    delete = " _()[]"
    delete_rules = s.maketrans("","",delete)
    s = s.translate(delete_rules)

    # step 2 : replace “” with ""
    to_be_replace = '“”'
    replace = '""'
    replace_dict = dict(zip(to_be_replace,replace))
    # print(replace_dict.items()) : dict_items([('“', '"'), ('”', '"')])

    # step 3 : add **END** before and after punctuation

    """
    The number of sentences in one line may be different
    in line pairs of source and target set.
    so I try to use "。!?" or ".!?" to split sentences.

    """

#    punctuation = "。,;!?()\"~「」"
    punctuation = "♪♫。!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    zh_list = s.strip("\n").split("\n")

    return zh_list

def clean_s_en(s):
    s = to_halfwidth(s)

    replace_dict = {}

    delete = "-()[]"
    for char in delete:
      replace_dict[char] = ""

    punctuation = "♪♫!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"
    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
    s = pattern.sub("**END**",s)

    en_list = s.strip("\n").split("\n")

    return en_list

In [ ]:
# pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
# result = pattern.sub("**END**","There are many people in U.S. w.r.t. in Taiwan.Thank you.")
# print(result)

In [169]:
def load_file(path,fuction):
  with open(path, "r") as f:
    data = f.read()
    return(fuction(data))
src_list = load_file(src_path,clean_s_en)
tgt_list = load_file(tgt_path,clean_s_zh)

In [ ]:
# with open(tgt_path, "r") as f:
#     data = f.readlines()
#     print(data[17])
# print(tgt_list[17])

In [170]:
def divide_by_END(s):
    list_s = []
    for line_string in s.strip("**END**").split("**END**"):
      if line_string not in [""," "]:
         list_s.append(line_string)
    return(list_s)

def devide_en_again(s,punctuation = ":;"):
    replace_dict = {}
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules_src = s.maketrans(replace_dict)
    new_s = divide_by_END(s.translate(replace_rules_src))
    return new_s
def remove_too_long(src_list,tgt_list,s_threshold = 400, t_threshold = 120):
    too_long_src = 0
    too_long_tgt = 0
    remove = False
    new_s = []
    new_t = []
    for i in range(len(src_list)):
      if ((len(src_list[i])>s_threshold)):
        remove = True
        too_long_src += 1
      if (len(tgt_list[i])>t_threshold):
        remove = True
        too_long_tgt += 1
      if remove == False:
        new_s.append(src_list[i])
        new_t.append(tgt_list[i])
      else :
        remove = False
    return(new_s,new_t,too_long_src,too_long_tgt)


In [171]:
def check_data_pairs(src_list,tgt_list):
    index = 0
    new_src_list = []
    new_tgt_list = []

    same = 0
    add_next = 0
    split_again = 0
    not_use = 0

    while(index < len(src_list)):

      src = divide_by_END(src_list[index])
      tgt = divide_by_END(tgt_list[index])
      # case 1 : src is as long as tgt , finished.
      if len(src) == len(tgt):
        new_src_list += src
        new_tgt_list += tgt
        same += 1
        index += 1

      else :
        # if it is not the last one : both src and tgt add next sentence
        if index != len(src_list)-1:
          src_add_next = divide_by_END(src_list[index] + src_list[index+1])
          tgt_add_next = divide_by_END(tgt_list[index] + tgt_list[index+1])
          # case 2 : src_add_next is as long as tgt_add_next , finished.
          if len(src_add_next) == len(tgt_add_next):
            new_src_list += src_add_next
            new_tgt_list += tgt_add_next
            add_next += 2
            index += 2

          # using new punctuation to divide tgt (english) sentence.
          else :
            src_add_next = devide_en_again(src_list[index] + src_list[index+1])
            # case 3 : src_add_next is as long as tgt_add_next , finished.
            if len(src_add_next) == len(tgt_add_next):
              new_src_list += src_add_next
              new_tgt_list += tgt_add_next
              split_again +=2
              index += 2

            # case 4 : sentence will not be used.
            else :
              not_use += 1
              # if to_do == 1 :
              #   print(index,src_add_next,tgt_add_next,len(src_add_next),len(tgt_add_next))
              index += 1

        # if it is the last one
        else :
          not_use += 1
          index += 1
    print(len(new_src_list))
    new_src_list,new_tgt_list,too_long_src,too_long_tgt = remove_too_long(new_src_list,new_tgt_list)
    print(index,same,add_next,split_again,not_use,too_long_src,too_long_tgt,len(new_src_list))

    return(new_src_list,new_tgt_list)

In [172]:
new_src_list,new_tgt_list = check_data_pairs(src_list,tgt_list)
# print(dataset[0][0],dataset[1][0])

389602
394066 350421 12284 2980 28381 677 605 388728


In [173]:
data_path = "/content/train_dev/data.txt"
label_path = "/content/train_dev/label.txt"
with open(data_path, "w") as f:
  f.write("\n".join(new_src_list))
with open(label_path, "w") as f:
  f.write("\n".join(new_tgt_list))

In [251]:
def get_max(l):
  exceed = 0
  maxize_sentence_length = 0
  for sentence in l:
    if len(sentence) > maxize_sentence_length:
      maxize_sentence_length = len(sentence)
  return(maxize_sentence_length)

data_info = {
    "data":{"path":data_path,"lang":"en","max_l":get_max(new_src_list)},
    "label":{"path":label_path,"lang":"zh","max_l":get_max(new_tgt_list)},
}
print(get_max(new_src_list),get_max(new_tgt_list))

400 120


In [175]:
!pip install sentencepiece

In [176]:
import sentencepiece as spm
vocab_size = 8000
def tokenized(data_or_label,data_info=data_info):
  spm.SentencePieceTrainer.train(
      input=data_info[data_or_label]["path"],
      model_prefix=f'spm{vocab_size}_{data_info[data_or_label]["lang"]}',
      vocab_size=vocab_size,
      character_coverage=1,
      model_type='unigram', # 'bpe' works as well
      input_sentence_size=400000,
      shuffle_input_sentence=True,
      normalization_rule_name='nmt_nfkc_cf',
      unk_id=1,
      bos_id=2,
      eos_id=3,
      pad_id=0,
  )

In [177]:
tokenized("data")
tokenized("label")

In [254]:
import sentencepiece as spm
import numpy as np
def bos_eos_padding(dataset,src_max,tgt_max):
  s_en = spm.SentencePieceProcessor(model_file="/content/spm8000_en.model")
  s_zh = spm.SentencePieceProcessor(model_file="/content/spm8000_zh.model")
  padding_src = []
  padding_tgt = []
  for src,tgt in dataset:
    s = s_en.encode(src, out_type=int)
    s = np.append([2],np.pad(s,(0, src_max-len(src)+1), constant_values = 0))
    s = np.append(s,[3])
    padding_src.append(s)

    t = s_zh.encode(tgt, out_type=int)
    t = np.append([2],np.pad(t,(0, tgt_max-len(tgt)+1), constant_values = 0))
    t = np.append(t,[3])
    padding_tgt.append(t)
  return(list(zip(padding_src,padding_tgt)))

In [233]:
# bos_eos_padding([([5,7],[6,5])],5,10)

In [259]:
import torch.utils.data as data
def data_set_preparing():
    src_set = []
    tgt_set = []

    with open("/content/train_dev/data.txt","r") as in_f :
      for line in in_f:
        src_set.append(line)
    with open("/content/train_dev/label.txt","r") as in_f :
      for line in in_f:
        tgt_set.append(line)

    dataset = list(zip(src_set,tgt_set))
    dataset = bos_eos_padding(dataset,data_info["data"]["max_l"],data_info["label"]["max_l"])
    train_set, valid_set = data.random_split(dataset,[0.9,0.1])
    print(train_set[0][0])

    with open("/content/train_dev/tokenized_train_data.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open("/content/train_dev/tokenized_valid_data.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open("/content/train_dev/tokenized_train_label.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")
    with open("/content/train_dev/tokenized_valid_label.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")
data_set_preparing()

[   2   23 1256   11   20  144   12 3390  108  138   12   17  211   68
  756  602   34   53 3843   34   53  293    6  172  471   34   53  864
  189  234   34   69   14   88   93    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [292]:
import torch
import numpy as np
from torch.utils.data import Dataset

class myDataset(Dataset):
  def __init__(self,mode):
    if mode == "train":
      self.data_path = "/content/train_dev/tokenized_train_data.txt"
      self.label_path = "/content/train_dev/tokenized_train_label.txt"
    elif mode == "val":
      self.data_path = "/content/train_dev/tokenized_valid_data.txt"
      self.label_path = "/content/train_dev/tokenized_valid_label.txt"

    data = []
    with open(self.data_path,"r") as f :
      d_l = f.readlines()
      for line in d_l:
        add = np.array(line.strip().split(" ")).astype(int)
        data = np.append(data,add)
    self.data = torch.LongTensor(data)

    label = []
    with open(self.label_path,"r") as f :
      l_l = f.readlines()
      for line in l_l:
        add = np.array(line.strip().split(" ")).astype(int)
        label = np.append(label,add)
    self.label = torch.LongTensor(np.array(label))


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.data, self.label


In [297]:
data = []
with open("/content/train_dev/tokenized_train_data.txt","r") as f :
  d_l = f.readlines()
  print(len(d_l))
  print(d_l[0].strip().split(" "))
  add = np.array(line.strip().split(" ")).astype(int)
data = np.append(data,add)

349856
['2', '23', '1256', '11', '20', '144', '12', '3390', '108', '138', '12', '17', '211', '68', '756', '602', '34', '53', '3843', '34', '53', '293', '6', '172', '471', '34', '53', '864', '189', '234', '34', '69', '14', '88', '93', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '

In [298]:
from torch.utils.data import DataLoader

batch_size = 400

train_loader = DataLoader(
  myDataset("train"),
  batch_size=batch_size,
  shuffle=True,
  num_workers=8,
  pin_memory=True,
)
valid_loader = DataLoader(
  myDataset("val"),
  batch_size=batch_size,
  num_workers=8,
  pin_memory=True,
)


KeyboardInterrupt: ignored

In [ ]:
# batch = {
#   "id": id, # 每個 example 的 id
#   "nsentences": len(samples), # batch size 句子數
#   "ntokens": ntokens, # batch size 字數
#   "net_input": {
#       "src_tokens": src_tokens, # 來源語言的序列
#       "src_lengths": src_lengths, # 每句話沒有 pad 過的長度
#       "prev_output_tokens": prev_output_tokens, # 上面提到右 shift 一格後的目標序列
#   },
#   "target": target, # 目標序列
# }

In [ ]:
# cuda_env = utils.CudaEnvironment()
# utils.CudaEnvironment.pretty_print_cuda_env_list([cuda_env])
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# # # HINT: transformer 架構
# # from fairseq.models.transformer import (
# #     TransformerEncoder,
# #     TransformerDecoder,
# # )

# def build_model(args, task):
#     """ 按照參數設定建置模型 """
#     src_dict, tgt_dict = task.source_dictionary, task.target_dictionary

#     # 詞嵌入
#     encoder_embed_tokens = nn.Embedding(len(src_dict), args.encoder_embed_dim, src_dict.pad())
#     decoder_embed_tokens = nn.Embedding(len(tgt_dict), args.decoder_embed_dim, tgt_dict.pad())

#     # 編碼器與解碼器
#     # TODO: 替換成 TransformerEncoder 和 TransformerDecoder
#     encoder = RNNEncoder(args, src_dict, encoder_embed_tokens)
#     decoder = RNNDecoder(args, tgt_dict, decoder_embed_tokens)

#     # 序列到序列模型
#     model = Seq2Seq(args, encoder, decoder)

#     # 序列到序列模型的初始化很重要 需要特別處理
#     def init_params(module):
#         from fairseq.modules import MultiheadAttention
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=0.02)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         if isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=0.02)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         if isinstance(module, MultiheadAttention):
#             module.q_proj.weight.data.normal_(mean=0.0, std=0.02)
#             module.k_proj.weight.data.normal_(mean=0.0, std=0.02)
#             module.v_proj.weight.data.normal_(mean=0.0, std=0.02)
#         if isinstance(module, nn.RNNBase):
#             for name, param in module.named_parameters():
#                 if "weight" in name or "bias" in name:
#                     param.data.uniform_(-0.1, 0.1)

#     # 初始化模型
#     model.apply(init_params)
#     return model

In [ ]:
# class Seq2Seq(FairseqEncoderDecoderModel):
#     def __init__(self, args, encoder, decoder):
#         super().__init__(encoder, decoder)
#         self.args = args

#     def forward(
#         self,
#         src_tokens,
#         src_lengths,
#         prev_output_tokens,
#         return_all_hiddens: bool = True,
#     ):
#         """
#         Run the forward pass for an encoder-decoder model.
#         """
#         encoder_out = self.encoder(
#             src_tokens, src_lengths=src_lengths, return_all_hiddens=return_all_hiddens
#         )
#         logits, extra = self.decoder(
#             prev_output_tokens,
#             encoder_out=encoder_out,
#             src_lengths=src_lengths,
#             return_all_hiddens=return_all_hiddens,
#         )
#         return logits, extra